# Project Market Share

## Imports

In [1]:
# utils
from utils.market_share import *

from typing import Dict, List, Any
import pickle
from tqdm import tqdm 

Sample segments: {'female_44_less': np.int64(1631), 'female_45_more': np.int64(1034), 'male_44_less': np.int64(1451), 'male_45_more': np.int64(884)}
Sample size: 5000
Weights: {'female_44_less': np.float64(1.005031010413465), 'female_45_more': np.float64(0.8480333014252863), 'male_44_less': np.float64(1.1635155138048476), 'male_45_more': np.float64(0.9000757667545914)}
Sum of the weights: 5000.0


## Calculate market shares

In [2]:
# create a dict best practice 
model_market_share: Dict[str, List] = {
    "Model 0" : [V_0, res_mod0, None, False],
    "Model 1" : [V_1, res_mod1, None, False],
    "Model 2" : [V_2, res_mod2, None, False],
    "Model 3" : [V_3, res_mod3, None, False],
    "Model 4" : [V_3, results_nested, biogeme_nested, True]
}
file_name = 'market_shares.pickle'
try:
    with open(file_name, 'rb') as f:
        all_market_shares = pickle.load(f)
        print(f'Market shares read from {file_name}')
except FileNotFoundError:
    # If the file is not found, calculate the market shares for each model
    all_market_shares: Dict[str, Any] = {
        model: market_share(utilities, results, model_biogeme, is_nested) 
        for model, (utilities, results, model_biogeme, is_nested) in tqdm(model_market_share.items())
    }
    # Save the calculated market shares to the pickle file
    with open(file_name, 'wb') as f:
        pickle.dump(all_market_shares, f)
    print(f'Market shares calculated and saved in {file_name}')



  0%|          | 0/5 [00:00<?, ?it/s]

         Prob. walk  Prob. cycle      Prob. PT    Prob. car
count  5.000000e+03  5000.000000  5.000000e+03  5000.000000
mean   1.704001e-01     0.029200  3.574000e-01     0.443000
std    1.629612e-01     0.018439  1.821333e-01     0.220800
min    1.903461e-13     0.000043  9.526819e-07     0.002286
25%    8.750002e-03     0.019479  2.575017e-01     0.278642
50%    1.257326e-01     0.025677  3.302956e-01     0.401025
75%    3.145519e-01     0.033997  4.215278e-01     0.594204
max    6.333856e-01     0.250870  9.970501e-01     0.999367


 20%|██        | 1/5 [01:00<04:03, 60.96s/it]

         Prob. walk   Prob. cycle     Prob. PT    Prob. car
count  5.000000e+03  5.000000e+03  5000.000000  5000.000000
mean   1.704008e-01  2.919996e-02     0.357400     0.443000
std    2.270605e-01  1.660377e-02     0.238669     0.213765
min    2.174021e-26  5.122602e-07     0.000627     0.000161
25%    8.570957e-05  1.832766e-02     0.185888     0.284840
50%    3.310869e-02  2.910390e-02     0.286574     0.464464
75%    3.151704e-01  3.868107e-02     0.441198     0.600420
max    8.677694e-01  1.640226e-01     0.999810     0.999295


 40%|████      | 2/5 [02:04<03:06, 62.19s/it]

...

         Prob. walk  Prob. cycle     Prob. PT    Prob. car
count  5.000000e+03  5000.000000  5000.000000  5000.000000
mean   1.704009e-01     0.029201     0.357401     0.442998
std    2.292483e-01     0.017832     0.245960     0.218750
min    1.155219e-30     0.000046     0.007444     0.001268
25%    2.714156e-05     0.017530     0.166425     0.280431
50%    2.740173e-02     0.026994     0.284779     0.457531
75%    3.187701e-01     0.038115     0.474462     0.606570
max    8.955295e-01     0.157479     0.998519     0.990473


 80%|████████  | 4/5 [04:23<01:07, 67.11s/it]

         Prob. walk  Prob. cycle     Prob. PT    Prob. car
count  5.000000e+03  5000.000000  5000.000000  5000.000000
mean   1.704053e-01     0.029198     0.358718     0.441679
std    2.262141e-01     0.013285     0.254006     0.222438
min    4.843372e-28     0.001327     0.005325     0.002272
25%    5.039167e-05     0.020409     0.155179     0.273727
50%    3.071888e-02     0.028397     0.283240     0.448481
75%    3.192087e-01     0.036473     0.496332     0.607476
max    8.397208e-01     0.080191     0.995338     0.974470


100%|██████████| 5/5 [05:43<00:00, 68.66s/it]

Market shares calculated and saved in market_shares.pickle


In [3]:
print(all_market_shares.keys())

dict_keys(['Model 0', 'Model 1', 'Model 2', 'Model 3', 'Model 4'])


In [4]:
print(all_market_shares["Model 0"])
print(all_market_shares["Model 1"])
print(all_market_shares["Model 2"])
print(all_market_shares["Model 3"])
print(all_market_shares["Model 4"])

{'Walking': IndicatorTuple(value=np.float64(0.16935243307249462), lower=np.float64(0.1599328760105841), upper=np.float64(0.1776773503878678)), 'Cycling': IndicatorTuple(value=np.float64(0.029289763131547717), lower=np.float64(0.026345566251124428), upper=np.float64(0.03237782973253703)), 'Public transportation': IndicatorTuple(value=np.float64(0.35834388719484866), lower=np.float64(0.34439408918920733), upper=np.float64(0.37305208691953284)), 'Car': IndicatorTuple(value=np.float64(0.443013916601109), lower=np.float64(0.4274760102996015), upper=np.float64(0.459206776913624))}
{'Walking': IndicatorTuple(value=np.float64(0.169125053350366), lower=np.float64(0.15881815951294362), upper=np.float64(0.1793852533931435)), 'Cycling': IndicatorTuple(value=np.float64(0.02917429367354091), lower=np.float64(0.023957860043166943), upper=np.float64(0.03601133355416872)), 'Public transportation': IndicatorTuple(value=np.float64(0.3598359886571484), lower=np.float64(0.3406186316941818), upper=np.float6